In [6]:
%cd COVID19-classifier/

/home/jupyter/COVID19-classifier


In [20]:
import os
from PIL import Image
import matplotlib.pyplot as plt
import os
import torch
import pandas as pd
import numpy as np
from torch import optim, nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, datasets
from transform import *
from utils import CustomCompose, CustomFolder
from models import *

In [21]:
DN = DenseNet121().cuda()
VGG = VGG16().cuda()

In [23]:
criterion = nn.BCELoss()
optimizer_DN = optim.Adam(DN.parameters(), lr=0.001)
optimizer_VGG = optim.Adam(VGG.parameters(), lr=0.001)

In [24]:
transform1 = transforms.Resize((224, 224))
transform2 = transforms.Compose([transforms.Resize((224, 224)),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.RandomRotation(10),
                                 transforms.ToTensor(),
                                 transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                                      std=[0.5, 0.5, 0.5])])
data = datasets.ImageFolder(root='dataset/train', transform=transform1)
raw_data = np.array([np.array(image[0], dtype="float") for image in data])
raw_minority_data = np.array([np.array(image[0], dtype="float") for image in data if image[1]==0])

In [25]:
transform_smote = CustomCompose([transforms.Resize((224, 224)),
                                 Smote(raw_minority_data, 5),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.RandomRotation(10),
                                 transforms.ToTensor(),
                                 transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                                      std=[0.5, 0.5, 0.5])])
data_smote = CustomFolder(root='dataset/train', transform=transform_smote)